In [17]:
# 0. before running this:
# create superuser, user, organization and password

# 1. setup user and organization

import escalateclient
import importlib

importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = "qai1"
password = "eBYvN4ELjrLXMDt"
client = escalateclient.ESCALATEClient(server_url, username, password)

# mnl should already be created
mnl_lab = client.get(endpoint="organization", data={"short_name": "MNL"})
assert len(mnl_lab) == 1
mnl_lab = mnl_lab[0]

{'short_name': 'MNL'}
GET: OK. Found 1 results


In [18]:
# 2. create materials identifiers and types
mids = []
for name in ["SMILES", "InChI", "Name", "IUPAC Name", "Label"]:
    mid = client.get_or_create(
        endpoint="material-identifier-def",
        data={"description": name}
    )
    mids += mid
assert len(mids) == 5
mid_smiles, mid_inchi, mid_name, mid_iupac, mid_label = mids

# we have 3 material types: solvent, ligand, and nano crystal
mt_types = []
for mt in ["Material Type: Solvent", "Material Type: Ligand", "Material Type: Nanocrystal"]:
    mt_type = client.get_or_create(endpoint="material-type", data={"description": mt})
    mt_types += mt_type
mt_solvent, mt_ligand, mt_nanocrystal = mt_types

{'description': 'SMILES'}
GET: OK. Found 1 results
{'description': 'InChI'}
GET: OK. Found 1 results
{'description': 'Name'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'IUPAC Name'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Label'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Material Type: Solvent'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Material Type: Ligand'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'Material Type: Nanocrystal'}
GET: OK. Found 0 results
POST: OK, returning new resource dict


In [21]:
# 3. add materials, then add them to inventory
# define add material function
def add_material(mi_values, mi_types, client, mt, material_description):
    mi_list = []
    for mi_value, mi_type in zip(mi_values, mi_types):
        mi = client.get_or_create(
            endpoint="material-identifier",
            data={
                "description": mi_value,
                # "full_description": mi_value,  # it seems that full_description will be automatically overwritten as "mi_type: mi_value"
                "material_identifier_def": mi_type["url"],
            },
        )[0]
        mi_list.append(mi)
    # I have to first check if this is already posted to the database
    m = client.get(endpoint="material", data={"description": material_description})
    assert len(m) <= 1
    if len(m) != 1:
        m_data={
            "description": material_description,
            "identifier":[mi["url"] for mi in mi_list],
            "material_class": "model",
            "material_type":[mt["url"],],
        }
        m = client.post(
            endpoint="material",
            data=m_data,
        )
    return m

import pandas as pd
# import ligands to materials
inventory_df = pd.read_csv("../inventory/inventory.csv", dtype={'LigandLabel': object})  # load Label as string
mi_defs = [mid_smiles, mid_iupac, mid_inchi, mid_label, mid_name]
for record in inventory_df.to_dict(orient="records"):
    mi_values = [record[k] for k in ['Canonical SMILES', 'IUPAC Name', 'InChI', 'LigandLabel', 'Name']]
    mi_values[3] = "LIGAND-" + mi_values[3]
    m = add_material(mi_values, mi_defs, client, mt_ligand, material_description=mi_values[3])

# now add solvent, data from https://pubchem.ncbi.nlm.nih.gov/compound/m-Xylene
solvent_mi_values = ["CC1=CC(=CC=C1)C", "1,3-xylene", "InChI=1S/C8H10/c1-7-4-3-5-8(2)6-7/h3-6H,1-2H3", 'SOLVENT-0000', 'm-xylene']
m_solvent = add_material(solvent_mi_values, mi_defs, client, mt_solvent, solvent_mi_values[3])
# also, the nanocrystal
perovskite_mi_defs = [mid_name,]
perovskite_mi_values = ["CsPbBr3 nanocrystal",]
m_perovskite = add_material(perovskite_mi_values, perovskite_mi_defs, client, mt_nanocrystal, perovskite_mi_values[0])


{'description': 'CCCCCCCC(=O)O', 'material_identifier_def': '73b38b61-c496-42c5-8f3b-ffb403608b95'}
GET: OK. Found 1 results
{'description': 'octanoic acid', 'material_identifier_def': '05b57ac2-9d2f-40d8-aa34-eaeab5c7475d'}
GET: OK. Found 1 results
{'description': 'InChI=1S/C8H16O2/c1-2-3-4-5-6-7-8(9)10/h2-7H2,1H3,(H,9,10)', 'material_identifier_def': '7b23b824-6088-4e47-b138-787ad57fb65d'}
GET: OK. Found 1 results
{'description': 'LIGAND-0000', 'material_identifier_def': '4dcae12f-a2c0-4214-b448-c0ac8d441a95'}
GET: OK. Found 1 results
{'description': 'Caprylic acid (octanoic acid)', 'material_identifier_def': 'a5812974-f219-477a-a8db-66f8c4410b79'}
GET: OK. Found 1 results
{'description': 'LIGAND-0000'}
GET: OK. Found 1 results
{'description': 'CCCCCCCCCCCC(=O)O', 'material_identifier_def': '73b38b61-c496-42c5-8f3b-ffb403608b95'}
GET: OK. Found 1 results
{'description': 'dodecanoic acid', 'material_identifier_def': '05b57ac2-9d2f-40d8-aa34-eaeab5c7475d'}
GET: OK. Found 1 results
{'de

In [22]:
# create a lab actor
lab_actor = client.get(endpoint="actor", data={"organization": mnl_lab["url"], "person__isnull": True, "systemtool_isnull": True})[0]

# use this actor to create our inventory
mnl_inventory = client.get_or_create(
    endpoint="inventory",
    data={
        "description": "NC Ligand Inventory",
        "lab": lab_actor["url"],
    }
)[0]

# add materials to our inventory
for m in client.get(endpoint="material", data={}):
    if m["material_type"][0] in [mt["url"] for mt in [mt_ligand, mt_solvent, mt_nanocrystal]]:
        client.get_or_create(endpoint="inventory-material", data={
            "inventory": mnl_inventory["url"],
            "material": m["url"],
            "description": m["description"],
        })

{'organization': '1a91690f-e6ea-44b1-bfb5-3b372ddbf4bd', 'person__isnull': True, 'systemtool_isnull': True}
GET: OK. Found 1 results
{'description': 'NC Ligand Inventory', 'lab': 'd2f50c3b-f919-4425-86b6-347159ba6802'}
GET: OK. Found 1 results
{}
GET: OK. Found 26 results
{'inventory': 'cf483a65-9bae-46fb-91d1-a02f47e49036', 'material': '9bf95843-5a49-4cb1-b0b0-a8da76f2f51c', 'description': 'LIGAND-0000'}
GET: OK. Found 1 results
{'inventory': 'cf483a65-9bae-46fb-91d1-a02f47e49036', 'material': 'ec2acc01-1eef-4257-bc0a-3715ec959107', 'description': 'LIGAND-0001'}
GET: OK. Found 1 results
{'inventory': 'cf483a65-9bae-46fb-91d1-a02f47e49036', 'material': '6c7b338c-ec48-4973-a54b-2b4f7251a336', 'description': 'LIGAND-0002'}
GET: OK. Found 1 results
{'inventory': 'cf483a65-9bae-46fb-91d1-a02f47e49036', 'material': '50c79b29-7f37-4b90-915b-951b35670393', 'description': 'SOLVENT-0000'}
GET: OK. Found 1 results
{'inventory': 'cf483a65-9bae-46fb-91d1-a02f47e49036', 'material': '163536b9-f6ac-4

In [23]:
# add vessels
column_order = "ABCDEFGH"
vessel_96well_plate = client.get_or_create(endpoint="vessel", data={
    "description": "96-well plate",
    "well_number": 96,
    "column_order": column_order,
})[0]

columns = "ABCDEFGH"
n_rows = 12
n_columns = len(columns)
for i_row in range(n_rows):
    for i_col in range(n_columns):
        well = client.get_or_create(
            endpoint="vessel",
            data={
                "description":  columns[i_col]+ "{0:02d}".format(int(i_row+1)),
                "well_number": 1,
                "parent": vessel_96well_plate["url"]
            }
        )

{'description': '96-well plate', 'well_number': 96, 'column_order': 'ABCDEFGH'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'A01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c3'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'B01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c3'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'C01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c3'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'D01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c3'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'E01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c3'}
GET: OK. Found 0 results
POST: OK, returning new resource dict
{'description': 'F01', 'well_number': 1, 'parent': '7c565943-9974-4c25-9048-b0eda64fa3c

In [ ]:
# add commonly used default values

zero_ul = {"value": 0, "unit": "ul", "type": "num", }
zero_M = {"value": 0, "unit": "M", "type": "num", }
zero_ul = client.get_or_create(endpoint="default-values", data={"description": "zero ul", "nominal_value": zero_ul, "actual_value": zero_ul, })[0]
zero_M = client.get_or_create(endpoint="default-values", data={"description": "zero M", "nominal_value": zero_M, "actual_value": zero_M, })[0]


In [ ]:
# add commonly used properties


for name in
client.get_or_create(endpoint="property-template", data={
    "property_def_class": "intrinsic",
    ""
})
